# Reinforcement learning for tic-tac-toe


In [1]:
import random
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.backend import reshape
from keras.utils.np_utils import to_categorical

ModuleNotFoundError: No module named 'keras'

In [3]:
def initBoard():
    board = [
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]
    ]
    return board

# Print the current state of the board
def printBoard(board):
    for i in range(len(board)):
        for j in range(len(board[i])):
            mark = ' '
            if board[i][j] == 1:
                mark = 'X'
            elif board[i][j] == 2:
                mark = 'O'
            if (j == len(board[i]) - 1):
                print(mark)
            else:
                print(str(mark) + "|", end='')
        if (i < len(board) - 1):
            print("-----")

# Get a list of valid moves (indices into the board)
def getMoves(board):
    moves = []
    for i in range(len(board)):
        for j in range(len(board[i])):
            if board[i][j] == 0:
                moves.append((i, j))
    return moves

# Declare a winner
#
# -1 = game not over
#  0 = draw
#  1 = 'X' wins (player 1)
#  2 = 'O' wins (player 2)
def getWinner(board):
    candidate = 0
    won = 0
    
    # Check rows
    for i in range(len(board)):
        candidate = 0
        for j in range(len(board[i])):
            
            # Make sure there are no gaps
            if board[i][j] == 0:
                break
            
            # Identify the front-runner
            if candidate == 0:
                candidate = board[i][j]
            
            # Determine whether the front-runner has all the slots
            if candidate != board[i][j]:
                break
            elif j == len(board[i]) - 1:
                won = candidate
    
    if won > 0:
        return won
    
    # Check columns
    for j in range(len(board[0])):
        candidate = 0
        for i in range(len(board)):
            
            # Make sure there are no gaps
            if board[i][j] == 0:
                break
            
            # Identify the front-runner
            if candidate == 0:
                candidate = board[i][j]
            
            # Determine whether the front-runner has all the slots
            if candidate != board[i][j]:
                break
            elif i == len(board) - 1:
                won = candidate
    
    if won > 0:
        return won
    
    # Check diagonals
    candidate = 0
    for i in range(len(board)):
        if board[i][i] == 0:
            break
        if candidate == 0:
            candidate = board[i][i]
        if candidate != board[i][i]:
            break
        elif i == len(board) - 1:
            won = candidate
    
    if won > 0:
        return won
    
    candidate = 0
    for i in range(len(board)):
        if board[2 - i][2 - i] == 0:
            break
        if candidate == 0:
            candidate = board[2 - i][2 - i]
        if candidate != board[2 - i][2 - i]:
            break
        elif i == len(board) - 1:
            won = candidate
    
    if won > 0:
        return won
    
    # Still no winner?
    if (len(getMoves(board)) == 0):
        # It's a draw
        return 0
    else:
        # Still more moves to make
        return -1

In [23]:
random.seed()

# Get best next move for the given player at the given board position
def bestMove(board, model, player, rnd=0):
    scores = []
    moves = getMoves(board) # returns valid move
    
    # Make predictions for each possible move
    for i in range(len(moves)):
        future = np.array(board)
        future[moves[i][0]][moves[i][1]] = player
        prediction = model.predict(future.reshape((-1, 9)))[0]
        if player == 1:
            winPrediction = prediction[1]
            lossPrediction = prediction[2]
        else:
            winPrediction = prediction[2]
            lossPrediction = prediction[1]
        drawPrediction = prediction[0]
        if winPrediction - lossPrediction > 0:
            scores.append(winPrediction - lossPrediction)
        else:
            scores.append(drawPrediction - lossPrediction)

    # Choose the best move with a random factor
    bestMoves = np.flip(np.argsort(scores))
    for i in range(len(bestMoves)):
        if random.random() * rnd < 0.5:
            return moves[bestMoves[i]]

    # Choose a move completely at random
    return moves[random.randint(0, len(moves) - 1)]

# Simulate a game
def simulateGame(p1=None, p2=None, rnd=0):
    history = []
    board = initBoard()
    playerToMove = 1
    
    while getWinner(board) == -1:
        
        # Chose a move (random or use a player model if provided)
        move = None
        if playerToMove == 1 and p1 != None:
            move = bestMove(board, p1, playerToMove, rnd)
        elif playerToMove == 2 and p2 != None:
            move = bestMove(board, p2, playerToMove, rnd)
        else:
            moves = getMoves(board)
            move = moves[random.randint(0, len(moves) - 1)]
        
        # Make the move
        board[move[0]][move[1]] = playerToMove
        
        # Add the move to the history
        history.append((playerToMove, move))
        
        # Switch the active player
        playerToMove = 1 if playerToMove == 2 else 2
        
    return history

# Simulate a game
history = simulateGame()
print(history)

[(1, (0, 0)), (2, (1, 1)), (1, (0, 2)), (2, (2, 0)), (1, (0, 1))]


In [8]:
# Reconstruct the board from the move list
def movesToBoard(moves):
    board = initBoard()
    for move in moves:
        player = move[0]
        coords = move[1]
        board[coords[0]][coords[1]] = player
    return board

board = movesToBoard(history)
printBoard(board)
print(getWinner(board))

 |O|O
-----
O| |X
-----
X|X|X
1


In [43]:
games = [simulateGame() for _ in range(1000)]

In [44]:
# Aggregate win/loss/draw stats for a player
def gameStats(games, player=1):
    stats = {"win": 0, "loss": 0, "draw": 0}
    for game in games:
        result = getWinner(movesToBoard(game))
        if result == -1:
            continue
        elif result == player:
            stats["win"] += 1
        elif result == 0:
            stats["draw"] += 1
        else:
            stats["loss"] += 1
    
    winPct = stats["win"] / len(games) * 100
    lossPct = stats["loss"] / len(games) * 100
    drawPct = stats["draw"] / len(games) * 100

    print("Results for player %d:" % (player))
    print("Wins: %d (%.1f%%)" % (stats["win"], winPct))
    print("Loss: %d (%.1f%%)" % (stats["loss"], lossPct))
    print("Draw: %d (%.1f%%)" % (stats["draw"], drawPct))

gameStats(games)
print()
gameStats(games, player=2)

Results for player 1:
Wins: 524 (52.4%)
Loss: 237 (23.7%)
Draw: 239 (23.9%)

Results for player 2:
Wins: 237 (23.7%)
Loss: 524 (52.4%)
Draw: 239 (23.9%)


In [5]:
def getModel():
    numCells = 9 # How many cells in a 3x3 tic-tac-toe board?
    outcomes = 3 # How many outcomes are there in a game? (draw, X-wins, O-wins)
    model = Sequential()
    model.add(Dense(200, activation='relu', input_shape=(9, )))
    model.add(Dropout(0.2))
    model.add(Dense(125, activation='relu'))
    model.add(Dense(75, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(25, activation='relu'))
    model.add(Dense(outcomes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
    return model
model = getModel()
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               2000      
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 125)               25125     
_________________________________________________________________
dense_2 (Dense)              (None, 75)                9450      
_________________________________________________________________
dropout_1 (Dropout)          (None, 75)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 25)                1900      
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 7

In [45]:
# Get a set of board states labelled by who eventually won that game
def gamesToWinLossData(games):
    X = []
    y = []
    for game in games:
        winner = getWinner(movesToBoard(game))
        for move in range(len(game)):
            X.append(movesToBoard(game[:(move + 1)]))
            y.append(winner)

    X = np.array(X).reshape((-1, 9))
    y = to_categorical(y)
    
    # Return an appropriate train/test split
    trainNum = int(len(X) * 0.8)
    return (X[:trainNum], X[trainNum:], y[:trainNum], y[trainNum:])

# Split out train and validation data
X_train, X_test, y_train, y_test = gamesToWinLossData(games)

In [46]:
nEpochs = 100
batchSize = 100
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=nEpochs, batch_size=batchSize)

Epoch 1/100
63/63 [==============================] - 0s 6ms/step - loss: 0.9128 - acc: 0.5842 - val_loss: 0.8813 - val_acc: 0.5761
Epoch 2/100
63/63 [==============================] - 0s 5ms/step - loss: 0.8458 - acc: 0.6078 - val_loss: 0.8826 - val_acc: 0.5876
Epoch 3/100
63/63 [==============================] - 0s 6ms/step - loss: 0.8273 - acc: 0.6171 - val_loss: 0.8779 - val_acc: 0.5761
Epoch 4/100
63/63 [==============================] - 0s 5ms/step - loss: 0.8107 - acc: 0.6256 - val_loss: 0.8809 - val_acc: 0.5876
Epoch 5/100
63/63 [==============================] - 0s 5ms/step - loss: 0.7897 - acc: 0.6326 - val_loss: 0.8979 - val_acc: 0.5786
Epoch 6/100
63/63 [==============================] - ETA: 0s - loss: 0.7855 - acc: 0.635 - 0s 5ms/step - loss: 0.7856 - acc: 0.6369 - val_loss: 0.8755 - val_acc: 0.5895
Epoch 7/100
63/63 [==============================] - 0s 5ms/step - loss: 0.7711 - acc: 0.6446 - val_loss: 0.9008 - val_acc: 0.5921
Epoch 8/100
63/63 [==========================

Epoch 63/100
63/63 [==============================] - 0s 5ms/step - loss: 0.6248 - acc: 0.7098 - val_loss: 1.0336 - val_acc: 0.5825
Epoch 64/100
63/63 [==============================] - 0s 5ms/step - loss: 0.6139 - acc: 0.7217 - val_loss: 1.0270 - val_acc: 0.5831
Epoch 65/100
63/63 [==============================] - 0s 5ms/step - loss: 0.6133 - acc: 0.7138 - val_loss: 1.0200 - val_acc: 0.5697
Epoch 66/100
63/63 [==============================] - 0s 5ms/step - loss: 0.6108 - acc: 0.7194 - val_loss: 0.9950 - val_acc: 0.5825
Epoch 67/100
63/63 [==============================] - 0s 5ms/step - loss: 0.6070 - acc: 0.7215 - val_loss: 1.0425 - val_acc: 0.5742
Epoch 68/100
63/63 [==============================] - 0s 5ms/step - loss: 0.6192 - acc: 0.7125 - val_loss: 1.0089 - val_acc: 0.5767
Epoch 69/100
63/63 [==============================] - 0s 5ms/step - loss: 0.6140 - acc: 0.7175 - val_loss: 1.0151 - val_acc: 0.5857
Epoch 70/100
63/63 [==============================] - 0s 5ms/step - loss: 0.

In [131]:
def convert_matrix(games):
    data_history = []
    for game in games:
        for move in game:
            var = [ move[1][1], move[1][0],move[0]]
            data_history.append(var)
    return data_history
    

In [63]:
flag = True

# START GAME

In [62]:
from tkinter import *
from tkinter import messagebox
import random as r

def button(frame):          #Function to define a button
    b=Button(frame,padx=1,bg="papaya whip",width=3,text="   ",font=('arial',60,'bold'),relief="sunken",bd=10)
    return b
def change_a():             #Function to change the operand for the next player
    global a
    for i in ['O','X']:
        if not(i==a):
            a=i
            break
def reset():                #Resets the game
    global a
    for i in range(3):
        for j in range(3):
                b[i][j]["text"]=" "
                b[i][j]["state"]=NORMAL
    a=r.choice(['O','X'])
    board = initBoard()
def check():                #Checks for victory or Draw
    for i in range(3):
            if(b[i][0]["text"]==b[i][1]["text"]==b[i][2]["text"]==a or b[0][i]["text"]==b[1][i]["text"]==b[2][i]["text"]==a):
                    messagebox.showinfo("Congrats!!","'"+get_player_name(a)+"' has won")
                    reset()
    if(b[0][0]["text"]==b[1][1]["text"]==b[2][2]["text"]==a or b[0][2]["text"]==b[1][1]["text"]==b[2][0]["text"]==a):
        messagebox.showinfo("Congrats!!","'"+get_player_name(a)+"' has won")
        reset()   
    elif(b[0][0]["state"]==b[0][1]["state"]==b[0][2]["state"]==b[1][0]["state"]==b[1][1]["state"]==b[1][2]["state"]==b[2][0]["state"]==b[2][1]["state"]==b[2][2]["state"]==DISABLED):
        messagebox.showinfo("Tied!!","The match ended in a draw")
        reset()

def get_player_name(a):
    player = ""
    if a=='O':
        player = "Computer"
    elif a=='X':
        player="Human"
    return player

def computer_turn():
    a='O'
    label=Label(text=get_player_name(a)+"'s Chance",font=('arial',20,'bold'))
    move = bestMove(board, model, 1)
    board[move[0]][move[1]] = 1
    row = move[0]
    col = move[1]
    print(row , col)
    b[row][col].config(text=a)
    check()
   
    
    
def click(row,col):
    a='X'
    label=Label(text=get_player_name(a)+"'s Chance",font=('arial',20,'bold'))
    b[row][col].config(text=a,state=DISABLED,disabledforeground=colour[a])
    board[row][col] = 2
    check()
    computer_turn()
    
###############   Main Program #################
root=Tk()                   #Window defined
root.title("Tic-Tac-Toe Human Vs Computer")   #Title given
a=r.choice(['O','X'])       #Two operators defined
colour={'O':"deep sky blue",'X':"lawn green"}
b=[[],[],[]]
board = initBoard()
for i in range(3):
        for j in range(3):
            b[i].append(button(root))
            b[i][j].config(command= lambda row=i,col=j:click(row,col))  
            b[i][j].grid(row=i,column=j)
label=Label(text=get_player_name(a)+"'s Chance",font=('arial',20,'bold'))
label.grid(row=3,column=0,columnspan=3)
root.mainloop()

2 1
0 1
1 0
2 0


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\musab anees\appdata\local\programs\python\python38\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-62-f188003eede2>", line 73, in <lambda>
    b[i][j].config(command= lambda row=i,col=j:click(row,col))
  File "<ipython-input-62-f188003eede2>", line 61, in click
    computer_turn()
  File "<ipython-input-62-f188003eede2>", line 45, in computer_turn
    move = bestMove(board, model, 1)
  File "<ipython-input-23-1d31a8642c44>", line 32, in bestMove
    return moves[random.randint(0, len(moves) - 1)]
  File "c:\users\musab anees\appdata\local\programs\python\python38\lib\random.py", line 248, in randint
    return self.randrange(a, b+1)
  File "c:\users\musab anees\appdata\local\programs\python\python38\lib\random.py", line 226, in randrange
    raise ValueError("empty range for randrange() (%d, %d, %d)" % (istart, istop, width))
ValueError: empty range for 